In [1]:
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Model
from keras.layers import Input
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import SGD,Adam
from keras.applications.resnet50 import ResNet50
from keras import backend as K

Using TensorFlow backend.


#Get data

In [2]:
num_classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_val = x_train[40000:50000,:,:,:]
y_val = y_train[40000:50000,:]

x_train = x_train[:40000,:,:,:]
y_train = y_train[:40000,:]

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')


170500096/170498071 [==============================] - 2s 0us/step
x_train shape: (40000, 32, 32, 3)
40000 train samples
10000 val samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

#Build model

In [4]:
K.set_image_data_format('channels_last')
# K.set_image_dim_ordering('tf')

image_input = Input(shape=(32, 32, 3))
model = ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()
x = model.get_layer('res5a_branch2a').input
x = GlobalAveragePooling2D(name='avg_pool')(x)
#    x = Dense(512, activation='relu',name='fc-1')(x)
x = Dropout(0.5)(x)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
custom_resnet_model = Model(inputs=image_input,outputs= out)














102858752/102853048 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________

In [5]:
learningrate=1e-3 
momentum=0.8
custom_resnet_model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=learningrate, momentum=momentum),
              metrics=['accuracy'])

custom_resnet_model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                   

#Train model

In [6]:
batch_trainsize=32 
batch_testsize=32 
nb_epoch = 20
hist = custom_resnet_model.fit(x_train, y_train, batch_size=batch_trainsize, epochs=nb_epoch, verbose=1, validation_data=(x_val, y_val))



Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 76s 2ms/step - loss: 0.3110 - acc: 0.8954 - val_loss: 0.2054 - val_acc: 0.9215
Epoch 2/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.2230 - acc: 0.9165 - val_loss: 0.1660 - val_acc: 0.9361
Epoch 3/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.1847 - acc: 0.9295 - val_loss: 0.1486 - val_acc: 0.9423
Epoch 4/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.1658 - acc: 0.9366 - val_loss: 0.1378 - val_acc: 0.9459
Epoch 5/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.1517 - acc: 0.9421 - val_loss: 0.1293 - val_acc: 0.9486
Epoch 6/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.1285 - acc: 0.9506 - val_loss: 0.1186 - val_acc: 0.9537
Epoch 8/20
40000/40000 [==============================] - 63s 2ms/step - loss: 0.1205 - acc: 0.9537 - val_loss: 0.1164 - val_a

#Evaluation of model with test data set

In [17]:
 (loss, accuracy) = custom_resnet_model.evaluate(x_test, y_test, batch_size=batch_testsize, verbose=1)
 print('loss: ',loss)
print('accuracy: ',accuracy)

10000/10000 [==============================] - 4s 384us/step
loss:  0.11885942187905311
accuracy:  0.9592599963188171


#Save model to drive

In [9]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
custom_resnet_model.save('gdrive/My Drive/activelaerning/homework1/model_rn50.h5')

In [0]:
from keras.models import load_model
model_loaded = load_model('gdrive/My Drive/activelaerning/homework1/model_rn50.h5')

In [16]:
 (loss, accuracy) = model_loaded.evaluate(x_test, y_test, batch_size=batch_testsize, verbose=1)
print('loss: ',loss)
print('accuracy: ',accuracy)

10000/10000 [==============================] - 4s 385us/step
loss:  0.11885942187905311
accuracy:  0.9592599963188171
